In [46]:
import pandas as pd
from scipy import stats
import statsmodels.stats.multitest as smm

In [2]:
df=pd.read_csv('gene_high_throughput_sequencing.csv')
df.head()

,Patient_id,Diagnosis,LOC643837,LOC100130417,SAMD11,NOC2L,KLHL17,PLEKHN1,C1orf170,HES4,...,CLIC2,RPS4Y1,ZFY,PRKY,USP9Y,DDX3Y,CD24,CYorf15B,KDM5D,EIF1AY
0,STT5425_Breast_001_normal,normal,1.257614,2.408148,13.368622,9.494779,20.880435,12.722017,9.494779,54.349694,...,4.761250,1.257614,1.257614,1.257614,1.257614,1.257614,23.268694,1.257614,1.257614,1.257614
1,STT5427_Breast_023_normal,normal,4.567931,16.602734,42.477752,25.562376,23.221137,11.622386,14.330573,72.445474,...,6.871902,1.815112,1.815112,1.815112,1.815112,1.815112,10.427023,1.815112,1.815112,1.815112
2,STT5430_Breast_002_normal,normal,2.077597,3.978294,12.863214,13.728915,14.543176,14.141907,6.232790,57.011005,...,7.096343,2.077597,2.077597,2.077597,2.077597,2.077597,22.344226,2.077597,2.077597,2.077597
3,STT5439_Breast_003_normal,normal,2.066576,8.520713,14.466035,7.823932,8.520713,2.066576,10.870009,53.292034,...,5.200770,2.066576,2.066576,2.066576,2.066576,2.066576,49.295538,2.066576,2.066576,2.066576
4,STT5441_Breast_004_normal,normal,2.613616,3.434965,12.682222,10.543189,26.688686,12.484822,1.364917,67.140393,...,11.227770,1.364917,1.364917,1.364917,1.364917,1.364917,23.627911,1.364917,1.364917,1.364917


## Task 1

In [9]:
def cut_df(df,diag):
    return df[df.Diagnosis==diag].drop(['Patient_id','Diagnosis'],axis=1)

In [12]:
df_n=cut_df(df,'normal')
df_e=cut_df(df,'early neoplasia')
df_c=cut_df(df,'cancer')

In [36]:
pvalue=pd.DataFrame(columns=['pvalue1','pvalue2'])
for col in df.columns[2:]:
    pvalue1=stats.ttest_ind(df_n[col], df_e[col], equal_var = False).pvalue
    pvalue2=stats.ttest_ind(df_e[col], df_c[col], equal_var = False).pvalue
    pvalue.loc[col]=[pvalue1,pvalue2]
pvalue.head()

,pvalue1,pvalue2
LOC643837,0.690766,0.413735
LOC100130417,0.000032,0.653429
SAMD11,0.060273,0.079556
NOC2L,0.826429,0.287581
KLHL17,0.049876,0.463292


In [43]:
a=0.05
print ("qty_pvalue1=",len(pvalue[pvalue.pvalue1<a]))
print ("qty_pvalue2=",len(pvalue[pvalue.pvalue2<a]))

qty_pvalue1= 1575
qty_pvalue2= 3490


## Task 2

In [66]:
rejected1,p_cor,a1,a2 = smm.multipletests(pvals=pvalue.pvalue1,alpha=a/2,method='holm')
rejected2,p_cor,a1,a2 = smm.multipletests(pvals=pvalue.pvalue2,alpha=a/2,method='holm')

In [67]:
pvalue['rejected1']=rejected1
pvalue['rejected2']=rejected2

In [68]:
pvalue.rejected1.value_counts()

False    15746
True         2
Name: rejected1, dtype: int64

In [69]:
pvalue.rejected2.value_counts()

False    15669
True        79
Name: rejected2, dtype: int64

In [91]:
def fold_change(C,T):
    if T>C:
        return T/C
    else: return -C/T

In [89]:
pvalue['n_mean']=df_n.mean()
pvalue['e_mean']=df_e.mean()
pvalue['c_mean']=df_c.mean()

In [93]:
pvalue['fc1'] = pvalue.apply(lambda row: fold_change(row['n_mean'], row['e_mean']), axis=1)
pvalue['fc2'] = pvalue.apply(lambda row: fold_change(row['e_mean'], row['c_mean']), axis=1)

In [103]:
print ("task2_qty1=",len(pvalue[(pvalue['rejected1']==False)&(abs(pvalue['fc1'])>1.5)]))
print ("task2_qty2=",len(pvalue[(pvalue['rejected2']==False)&(abs(pvalue['fc2'])>1.5)]))

task2_qty1= 259
task2_qty2= 994


## Task 3

In [105]:
rejected1,p_cor,a1,a2 = smm.multipletests(pvals=pvalue.pvalue1,alpha=a/2,method='fdr_bh')
rejected2,p_cor,a1,a2 = smm.multipletests(pvals=pvalue.pvalue2,alpha=a/2,method='fdr_bh')
pvalue['fdr_bh_rejected1']=rejected1
pvalue['fdr_bh_rejected2']=rejected2

In [106]:
pvalue

,pvalue1,pvalue2,rejected1,rejected2,n_mean,e_mean,c_mean,fc1,fc2,fdr_bh_rejected1,fdr_bh_rejected2
LOC643837,1.000000,1.000000,True,True,2.681277,2.510894,2.186060,-1.067858,-1.148593,False,False
LOC100130417,0.000032,0.653429,False,False,4.368497,8.721781,8.190456,1.996517,-1.064871,False,False
SAMD11,0.060273,0.079556,False,False,15.159566,18.531325,23.692614,1.222418,1.278517,False,False
NOC2L,0.826429,0.287581,False,False,15.374351,15.071854,16.468034,-1.020070,1.092635,False,False
KLHL17,0.049876,0.463292,False,False,21.459886,24.152469,25.035813,1.125471,1.036574,False,False
PLEKHN1,0.144267,0.007681,False,False,9.374537,10.793160,13.870940,1.151327,1.285160,False,False
C1orf170,0.329108,0.481306,False,False,8.105567,9.009031,9.677439,1.111462,1.074193,False,False
HES4,0.023712,0.578830,False,False,58.080871,64.670644,67.162763,1.113459,1.038536,False,False
ISG15,0.240148,0.000740,False,False,34.983861,37.296084,91.706112,1.066094,2.458867,False,True
AGRN,0.037933,0.712687,False,False,62.870812,75.707467,73.359613,1.204175,-1.032005,False,False


In [107]:
print ("task3_qty1=",len(pvalue[(pvalue['fdr_bh_rejected1']==False)&(abs(pvalue['fc1'])>1.5)]))
print ("task3_qty2=",len(pvalue[(pvalue['fdr_bh_rejected2']==False)&(abs(pvalue['fc2'])>1.5)]))

task3_qty1= 257
task3_qty2= 547
